In [4]:
import pandas as pd

In [ ]:
data=pd.read_pickle("24all_patents_atleat1000.pkl")

In [6]:
data.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
0,574528.0,307102421.0,1985-04-10,EP,Y,"[(574528, 1.999717), (9306402, 1.9822814), (84...",2000
1,574586.0,387396319.0,1987-06-08,EP,Y,"[(574586, 1.9997215), (63363761, 1.97839), (83...",2000
2,574660.0,387398325.0,1986-08-04,EP,Y,"[(574660, 1.9997239), (6240233, 1.9768794), (2...",1658
3,1226456.0,303518198.0,1991-08-29,EP,Y,"[(1226456, 1.9997258), (4856238, 1.9822338), (...",2000
4,1226485.0,302640285.0,1992-08-12,EP,Y,"[(1226485, 1.9996202), (6328120, 1.987576), (2...",2000


In [7]:
data['docdb_family_id']

0           574528.0
1           574586.0
2           574660.0
3          1226456.0
4          1226485.0
             ...    
229095    27494163.0
229096    27524585.0
229097    27542360.0
229098    27565511.0
229099    30128771.0
Name: docdb_family_id, Length: 229100, dtype: float64

In [8]:
import os
from dotenv import load_dotenv

In [9]:
import mysql.connector

In [10]:
load_dotenv()
db_config = {
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_DATABASE')
}

In [ ]:
#querying sql database to fetch citations
conn = mysql.connector.connect(**db_config)
family_ids=tuple(data['docdb_family_id'])
cursor = conn.cursor(dictionary=True)
query = f"""
SELECT    
	docdb_family_id,
    JSON_ARRAYAGG(cited_docdb_family_id) AS cited_docdb_family_id
FROM
    patstat_s24.tls228_docdb_fam_citn
WHERE
    docdb_family_id IN {family_ids}
GROUP BY 
    docdb_family_id;
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [12]:
table2=pd.DataFrame(results)

In [13]:
table2.head()

,docdb_family_id,cited_docdb_family_id
0,574528,"[5970115, 6025659, 6695018, 8626155, 9303184, ..."
1,574586,"[2979913, 3291893, 6281119, 9513207, 21967964,..."
2,574660,"[5956195, 11260528, 22181831, 22437920, 226429..."
3,1226456,"[6248581, 8808809, 9247749, 9270571, 22049919,..."
4,1226485,"[6206849, 6328120, 20427850]"


In [14]:
len(table2)

228198

In [15]:
data.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
0,574528.0,307102421.0,1985-04-10,EP,Y,"[(574528, 1.999717), (9306402, 1.9822814), (84...",2000
1,574586.0,387396319.0,1987-06-08,EP,Y,"[(574586, 1.9997215), (63363761, 1.97839), (83...",2000
2,574660.0,387398325.0,1986-08-04,EP,Y,"[(574660, 1.9997239), (6240233, 1.9768794), (2...",1658
3,1226456.0,303518198.0,1991-08-29,EP,Y,"[(1226456, 1.9997258), (4856238, 1.9822338), (...",2000
4,1226485.0,302640285.0,1992-08-12,EP,Y,"[(1226485, 1.9996202), (6328120, 1.987576), (2...",2000


In [16]:
len(data)

229100

In [17]:
229100-228198

902

In [18]:
#902 families for which citations were not found

In [19]:
data_with_citations = data.merge(table2, on='docdb_family_id', how='inner')

In [20]:
data_with_citations.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents,cited_docdb_family_id
0,574528.0,307102421.0,1985-04-10,EP,Y,"[(574528, 1.999717), (9306402, 1.9822814), (84...",2000,"[5970115, 6025659, 6695018, 8626155, 9303184, ..."
1,574586.0,387396319.0,1987-06-08,EP,Y,"[(574586, 1.9997215), (63363761, 1.97839), (83...",2000,"[2979913, 3291893, 6281119, 9513207, 21967964,..."
2,574660.0,387398325.0,1986-08-04,EP,Y,"[(574660, 1.9997239), (6240233, 1.9768794), (2...",1658,"[5956195, 11260528, 22181831, 22437920, 226429..."
3,1226456.0,303518198.0,1991-08-29,EP,Y,"[(1226456, 1.9997258), (4856238, 1.9822338), (...",2000,"[6248581, 8808809, 9247749, 9270571, 22049919,..."
4,1226485.0,302640285.0,1992-08-12,EP,Y,"[(1226485, 1.9996202), (6328120, 1.987576), (2...",2000,"[6206849, 6328120, 20427850]"


In [ ]:
#storing patents along with citaions
data_with_citations.to_pickle("25patents_with_citations.pkl")

In [22]:
len(data_with_citations)

228198

In [ ]:
#patents for which no citation was found
result_df = data.merge(table2, on='docdb_family_id', how='left', indicator=True)
result_df.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents,cited_docdb_family_id,_merge
0,574528.0,307102421.0,1985-04-10,EP,Y,"[(574528, 1.999717), (9306402, 1.9822814), (84...",2000,"[5970115, 6025659, 6695018, 8626155, 9303184, ...",both
1,574586.0,387396319.0,1987-06-08,EP,Y,"[(574586, 1.9997215), (63363761, 1.97839), (83...",2000,"[2979913, 3291893, 6281119, 9513207, 21967964,...",both
2,574660.0,387398325.0,1986-08-04,EP,Y,"[(574660, 1.9997239), (6240233, 1.9768794), (2...",1658,"[5956195, 11260528, 22181831, 22437920, 226429...",both
3,1226456.0,303518198.0,1991-08-29,EP,Y,"[(1226456, 1.9997258), (4856238, 1.9822338), (...",2000,"[6248581, 8808809, 9247749, 9270571, 22049919,...",both
4,1226485.0,302640285.0,1992-08-12,EP,Y,"[(1226485, 1.9996202), (6328120, 1.987576), (2...",2000,"[6206849, 6328120, 20427850]",both


In [24]:
data_no_citations=result_df[result_df['_merge']=="left_only"]

In [25]:
data_no_citations.drop(columns='_merge',inplace=True)

/tmp/ipykernel_665804/3604542973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_citations.drop(columns='_merge',inplace=True)


In [26]:
data_no_citations.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents,cited_docdb_family_id
17,1340681.0,300828810.0,1991-12-21,EP,Y,"[(6447919, 1.9997177), (9332406, 1.9764507), (...",2000,NaN
834,3503822.0,305636650.0,1987-04-16,EP,N,"[(4292021, 1.9803829), (5942358, 1.9782984), (...",2000,NaN
1022,3510361.0,310256370.0,1980-03-07,EP,N,"[(3510361, 1.9996486), (27560443, 1.9855101), ...",2000,NaN
1180,3515298.0,310056793.0,1981-04-01,EP,N,"[(3515298, 1.999682), (22844228, 1.9752696), (...",2000,NaN
1982,3558565.0,288121670.0,2002-01-10,EP,N,"[(3558565, 1.9997213), (7925896, 1.9898684), (...",2000,NaN


In [27]:
len(data_no_citations)

902

In [ ]:
data_no_citations.reset_index(drop=True).to_pickle("25patents_no_citations.pkl")

In [29]:
len(data_with_citations)

228198

In [30]:
228198+902

229100